# Creating static web-sites using AWS S3 Object Storage

You should first have
- installed the awscli package to provide the aws command and also
- configured either
  - the ~/.aws/configure file with your AWS account credentials or
  - created a sourceable ~/.aws/credentials.rc (can be in any location) file

```
> cat ~/.aws/credentials.rc

export AWS_ACCESS_KEY_ID="<your-access-key>"
export AWS_SECRET_ACCESS_KEY="<your-secret-access-key>"
export AWS_DEFAULT_REGION=us-west-1
```

If you have chosen to use an rc file, source it as ```source <your-aws-credentials-rc-file>```, e.g.

In [9]:
. ~/.aws/credentials

We can now use the aws cli utility to access S3 commands.

Let's investigate the available commands with ```aws s3 help```

In [10]:
aws s3 help 

S3()                                                                      S3()



NAME
       s3 -

DESCRIPTION
       This  section  explains  prominent concepts and notations in the set of
       high-level S3 commands provided.

   Path Argument Type
       Whenever using a command, at least one path argument must be specified.
       There are two types of path arguments: LocalPath and S3Uri.

       LocalPath: represents the path of a local file or directory.  It can be
       written as an absolute path or relative path.

       S3Uri: represents the location of a S3 object, prefix, or bucket.  This
       must  be  written in the form s3://mybucket/mykey where mybucket is the
       specified S3 bucket, mykey is the specified S3 key.  The path  argument
       must  begin with s3:// in order to denote that the path argument refers
       to a S3 object. Note that prefixes are separated  by  forward  slashes.
       For  example, if the S3 object myobject had the prefix myprefix,

S3 is organized into buckets, each of which contains a hierarchy of named objects.

The bucket name itself must be globally **unique** - across all AWS accounts not just yours.

Let's see if we have any buckets of our own using the ```aws s3 ls``` command.
You won't have any buckets if you just created your account.

In [11]:
aws s3 ls

Let's create our own bucket, we use the 'mb' or *make bucket* command to do this as
    ```aws s3 mb s3://<my-bucket-name>```
    
Note that we always address buckets in URL form ```s3://<bucket>```

In [12]:
aws s3 mb s3://mybucket

make_bucket failed: s3://mybucket An error occurred (BucketAlreadyExists) when calling the CreateBucket operation: The requested bucket name is not available. The bucket namespace is shared by all users of the system. Please select a different name and try again.


: 1

Of course someone got there first with our "mybucket" name !

So create a **unique** name, e.g. for myself:

In [13]:
aws s3 mb s3://mjbright-static-site

make_bucket: mjbright-static-site


In [14]:
aws s3 ls

2019-01-25 21:33:37 mjbright-static-site


That was lucky !

Now let's add some files to our bucket.

We can do this with the cp or sync commands.

Let's create an HTML index file and copy this into our bucket.

In [15]:
mkdir -p website;

cat > website/index.html <<EOF
<html>
<body>
    <h1> My first amazing web site !! </h1>
</body>
</html>
EOF

ls -al website/index.html

-rw-rw-r-- 1 user1 user1 74 Jan 25 21:33 website/index.html


In [16]:
aws s3 sync ./website s3://mjbright-static-site

upload: website/index.html to s3://mjbright-static-site/index.html


In [17]:
aws s3 ls s3://mjbright-static-site

2019-01-25 21:33:55         74 index.html


So it seems we have created a static web site.

We can use the handy S3 command website to declare that this is a website.

In [18]:
aws s3 website s3://mjbright-static-site --index-document index.html

The site should be available at http://<bucketname>.s3-website-<region>.amazonaws.com, which in this case would be:
    
    http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/
    
However, if we visit that web page we will get an error telling us that we cannot access the page

In [19]:
wget -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

--2019-01-25 21:34:20--  http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html
Resolving mjbright-static-site.s3-website-us-west-1.amazonaws.com (mjbright-static-site.s3-website-us-west-1.amazonaws.com)... 52.219.24.172
Connecting to mjbright-static-site.s3-website-us-west-1.amazonaws.com (mjbright-static-site.s3-website-us-west-1.amazonaws.com)|52.219.24.172|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2019-01-25 21:34:20 ERROR 403: Forbidden.



: 8

**NOTE**: The following steps using the AWS console can be avoided by

For more information see "awscli enabling of website hosting":  http://notes.webutvikling.org/add-s3-bucket-using-awscli-example/



In [21]:
BUCKET=mjbright-static-site

# NOT NEEDED AS WE ALREADY CREATED THE BUCKET:
# aws s3api create-bucket --bucket $BUCKET --region us-west-1 --acl public-read --create-bucket-configuration LocationConstraint=us-west-1

# NOT NEEDED as copy already performed:
# aws s3 cp --recursive website/ s3://$BUCKET/

curl https://raw.githubusercontent.com/tomfa/aws-policies/master/s3-bucket-public-read.json > /tmp/s3-bucket-public-read.json

cat /tmp/s3-bucket-public-read.json




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   236  100   236    0     0   1380      0 --:--:-- --:--:-- --:--:--  1380
{
  "Version":"2012-10-17",
  "Statement":[{
    "Sid":"PublicReadGetObject",
        "Effect":"Allow",
      "Principal": "*",
      "Action":["s3:GetObject"],
      "Resource":["arn:aws:s3:::[[YOUR-BUCKET-NAME]]/*"
      ]
    }
  ]
}

In [22]:
sed "s/\[\[YOUR-BUCKET-NAME\]\]/$BUCKET/g" /tmp/s3-bucket-public-read.json > s3.json
cat s3.json
aws s3api put-bucket-policy --bucket $BUCKET --policy file://s3.json

{
  "Version":"2012-10-17",
  "Statement":[{
    "Sid":"PublicReadGetObject",
        "Effect":"Allow",
      "Principal": "*",
      "Action":["s3:GetObject"],
      "Resource":["arn:aws:s3:::mjbright-static-site/*"
      ]
    }
  ]
}

In [24]:
http http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html 

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 74
Content-Type: text/html
Date: Fri, 25 Jan 2019 21:41:55 GMT
ETag: "a891baebfc13d226edc4eb4e908cf652"
Last-Modified: Fri, 25 Jan 2019 21:33:55 GMT
Server: AmazonS3
x-amz-id-2: 2kczFb8kdg8e4ecMKLiLFF73Tvy4TVp6uolBH5MdbaNYxeaBjJGSBJbX15iHgQqUjSiEz97+Hig=
x-amz-request-id: 463C085A1474ED01

<html>
<body>
    <h1> My first amazing web site !! </h1>
</body>
</html>



**NOTE: THESE GRAPHICAL/CONSOLE STEPS ARE NO LONGER NEEDED IF ABOVE policy commands used**

This information kept here for reference.

If the above request succeeded you can skip to section '**Static-site generators**' below.

_____________________________________________________________________________________________

In fact we first need to enable website hosting from the bucket **and** enable public access to the index.html file.

We can do this via the AWS Console.

Connect to the console with your credentials and then navigate to https://s3.console.aws.amazon.com/s3/buckets/.


![](images/BucketProperties.JPG)

You should see your bucket listed here.

Click on the line (not on the bucket name which is a link, but under the Access or Region column) to see the following dropdown menu

![](images/BucketProperties-BeforeWebHostingEnabled.JPG)

Click on "Enable Web hosting" and you should see:

<!-- ![](images/BucketProperties-Settings-EnableWebsiteHosting.JPG) -->

![](images/BucketProperties-Settings-Enabled_WebsiteHosting.JPG)

But we still cannot access our site as we need to enable public access to the index.html file.

Click on the bucket name to be taken to a list of files in the bucket, select the index.html file and then "*Make Public*" in the dropdown "*Actions*" menu:

![](images/Make_index_public.JPG)

You now should be able to access your site using a browser, or from the command-line:

In [25]:
http http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 74
Content-Type: text/html
Date: Fri, 25 Jan 2019 21:44:32 GMT
ETag: "a891baebfc13d226edc4eb4e908cf652"
Last-Modified: Fri, 25 Jan 2019 21:33:55 GMT
Server: AmazonS3
x-amz-id-2: r7BnsjmdWZMO+OwLbaSEIDj8xRfFK7J2EJy6nrLn3ibLjW6rf1DThFNH3LnytA3bpyqRNZ2HDZQ=
x-amz-request-id: 6BE5077594B573B8

<html>
<body>
    <h1> My first amazing web site !! </h1>
</body>
</html>



## Static-site generators

Let's now create something a little more like a website.

### 1. Pelican a Python Module
We'll use the Pelican command (installed as a Python module).


In [26]:
pelican -o PELICAN -q

cp -a PELICAN/* website/

aws s3 sync website/ s3://mjbright-static-site/

Done: Processed 0 articles, 0 drafts, 0 pages, 0 hidden pages and 0 draft pages in 0.07 seconds.
upload: website/archives.html to s3://mjbright-static-site/archives.html            
upload: website/images/AWS-SIGNUP.JPG to s3://mjbright-static-site/images/AWS-SIGNUP.JPG
upload: website/authors.html to s3://mjbright-static-site/authors.html                
upload: website/categories.html to s3://mjbright-static-site/categories.html          
upload: website/theme/css/reset.css to s3://mjbright-static-site/theme/css/reset.css
upload: website/images/BucketProperties-Settings-EnableWebsiteHosting.JPG to s3://mjbright-static-site/images/BucketProperties-Settings-EnableWebsiteHosting.JPG
upload: website/theme/css/pygment.css to s3://mjbright-static-site/theme/css/pygment.css
upload: website/images/BucketProperties-Settings-Enabled_WebsiteHosting.JPG to s3://mjbright-static-site/images/BucketProperties-Settings-Enabled_WebsiteHosting.JPG
upload: website/theme/css/main.css to s3://mjbright-sta

We can now test our new website: using wget, curl or http(ie) or using a browser at http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

In [27]:
http http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 1486
Content-Type: text/html
Date: Fri, 25 Jan 2019 21:45:31 GMT
ETag: "70ec20e1a2544e5eba9aa224e3b607f6"
Last-Modified: Fri, 25 Jan 2019 21:45:19 GMT
Server: AmazonS3
x-amz-id-2: ehBTFGAc6bUlc53B7VmUfM+QTJnSn+HN7wU4tXEcERzGM75sVJPZDVXD7BvjjxXqM9MD/rHWhNI=
x-amz-request-id: 4026EC8E31BF98D1

<!DOCTYPE html>
<html lang="en">
<head>
        <meta charset="utf-8" />
        <title>A Pelican Blog</title>
        <link rel="stylesheet" href="/theme/css/main.css" />
        <link href="/feeds/all.atom.xml" type="application/atom+xml" rel="alternate" title="A Pelican Blog Atom Feed" />
</head>

<body id="index" class="home">
        <header id="banner" class="body">
                <h1><a href="/">A Pelican Blog </a></h1>
                <nav><ul>
                </ul></nav>
      

http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

### 2. Hugo - a static site generator/CMS written in Go

Let's create a site using Hugo:
```hugo new site <sitename>```

In [28]:
hugo new site quickstart

cd quickstart

Congratulations! Your new Hugo site is created in /home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/quickstart.

Just a few more steps and you're ready to go:

1. Download a theme into the same-named folder.
   Choose a theme from https://themes.gohugo.io/, or
   create your own with the "hugo new theme <THEMENAME>" command.
2. Perhaps you want to add some content. You can add single files
   with "hugo new <SECTIONNAME>/<FILENAME>.<FORMAT>".
3. Start the built-in live server via "hugo server".

Visit https://gohugo.io/ for quickstart guide and full documentation.


Now let's install the ananke theme

In [36]:
git init
git submodule add https://github.com/budparr/gohugo-theme-ananke.git themes/ananke
echo 'theme = "ananke"' >> config.toml

Initialized empty Git repository in /home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/quickstart/.git/
Cloning into '/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/quickstart/themes/ananke'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1234 (delta 16), reused 18 (delta 7), pack-reused 1196
Receiving objects: 100% (1234/1234), 2.61 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (656/656), done.


and now
- create the site with this theme
- copy the files into our website directory
- re-sync to our S3 bucket

In [37]:
pwd
hugo

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/quickstart
25lBuilding sites … 
                   | EN  
+------------------+----+
  Pages            |  7  
  Paginator pages  |  0  
  Non-page files   |  0  
  Static files     |  3  
  Processed images |  0  
  Aliases          |  0  
  Sitemaps         |  1  
  Cleaned          |  0  

Total in 8 ms


In [38]:
ls -al public/

total 40
drwxrwxr-x  6 user1 user1 4096 Jan 25 21:57 .
drwxrwxr-x 11 user1 user1 4096 Jan 25 21:56 ..
-rw-rw-r--  1 user1 user1 2071 Jan 25 21:57 404.html
drwxrwxr-x  2 user1 user1 4096 Jan 25 21:57 categories
drwxrwxr-x  4 user1 user1 4096 Jan 25 21:56 dist
drwxrwxr-x  2 user1 user1 4096 Jan 25 21:56 images
-rw-rw-r--  1 user1 user1 2495 Jan 25 21:57 index.html
-rw-rw-r--  1 user1 user1  468 Jan 25 21:57 index.xml
-rw-rw-r--  1 user1 user1  437 Jan 25 21:57 sitemap.xml
drwxrwxr-x  2 user1 user1 4096 Jan 25 21:57 tags


In [40]:
cd ..

cp -a quickstart/public/* website/
aws s3 sync website/ s3://mjbright-static-site/

upload: website/404.html to s3://mjbright-static-site/404.html                      
upload: website/categories/index.xml to s3://mjbright-static-site/categories/index.xml
upload: website/dist/js/app.3fc0f988d21662902933.js to s3://mjbright-static-site/dist/js/app.3fc0f988d21662902933.js
upload: website/categories/index.html to s3://mjbright-static-site/categories/index.html
upload: website/tags/index.html to s3://mjbright-static-site/tags/index.html
upload: website/dist/css/app.955516233bcafa4d2a1c13cea63c7b50.css to s3://mjbright-static-site/dist/css/app.955516233bcafa4d2a1c13cea63c7b50.css
upload: website/index.html to s3://mjbright-static-site/index.html
upload: website/index.xml to s3://mjbright-static-site/index.xml  
upload: website/sitemap.xml to s3://mjbright-static-site/sitemap.xml
upload: website/images/gohugo-default-sample-hero-image.jpg to s3://mjbright-static-site/images/gohugo-default-sample-hero-image.jpg
upload: website/tags/index.xml to s3://mjbright-static-site/tags

We can now test our new website: using wget, curl or http(ie) or using a browser at http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

In [41]:
http http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 2495
Content-Type: text/html
Date: Fri, 25 Jan 2019 21:57:44 GMT
ETag: "0e015b35a60e4c5773d0cd12e57b5fc8"
Last-Modified: Fri, 25 Jan 2019 21:57:30 GMT
Server: AmazonS3
x-amz-id-2: L/bUXP5g8KE7KJUlX9c4+12xiLGE49MajgayUU3j0ro3LPkaZU2joyzwvlxigZQneoLO8TTyI18=
x-amz-request-id: C392A7553E4EB4CE

<!DOCTYPE html>
<html lang="en-us">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    
    <title>My New Hugo Site </title>
    <meta name="HandheldFriendly" content="True">
    <meta name="MobileOptimized" content="320">

    <meta name="viewport" content="width=device-width,minimum-scale=1">
    <meta name="generator" content="Hugo 0.53" />
    
    
      <META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW">
    

    
    
      <li

Visit http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

### 3. create-react-app

create-react-app is an npm module created by Facebook allowing to quickly create a skeleton React app.

In [ ]:
create-react-app my-react-app
#cd my-react-app/; npm start
#ls -al my-react-app/public/

In [60]:
cd my-react-app; npm run build
ls -al public/


> my-react-app@0.1.0 build /home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/my-react-app
> react-scripts build

▀ ╢░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░╟
Creating an optimized production build...
Compiled successfully.

File sizes after gzip:

  34.71 KB  build/static/js/1.fa92c112.chunk.js
  763 B     build/static/js/runtime~main.229c360f.js
  716 B     build/static/js/main.a7b1c78b.chunk.js
  510 B     build/static/css/main.8cc6a47e.chunk.css

The project was built assuming it is hosted at the server root.
You can control this with the homepage field in your package.json.
For example, add this to build it for GitHub Pages:

  "homepage" : "http://myname.github.io/myapp",

The build folder is ready to be deployed.
You may serve it with a static server:

  npm install -g serve
  serve -s build

Find out more about deployment here:

  http://bit.ly/CRA-deploy

▀ ╢░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░

In [63]:
ls -altr build

total 36
drwxrwxr-x 6 user1 user1 4096 Jan 25 22:12 ..
-rwxrwxr-x 1 user1 user1  306 Jan 25 22:13 manifest.json
-rwxrwxr-x 1 user1 user1 3870 Jan 25 22:13 favicon.ico
drwxrwxr-x 3 user1 user1 4096 Jan 25 22:13 .
drwxrwxr-x 5 user1 user1 4096 Jan 25 22:13 static
-rw-rw-r-- 1 user1 user1 1041 Jan 25 22:13 service-worker.js
-rw-rw-r-- 1 user1 user1  606 Jan 25 22:13 precache-manifest.16ab4e73d949e217b5bb10bd22a96200.js
-rw-rw-r-- 1 user1 user1 2062 Jan 25 22:13 index.html
-rw-rw-r-- 1 user1 user1  779 Jan 25 22:13 asset-manifest.json


In [64]:
cd ..; pwd
rm -rf website/*
cp -av my-react-app/build/* website/

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda
'my-react-app/build/asset-manifest.json' -> 'website/asset-manifest.json'
'my-react-app/build/favicon.ico' -> 'website/favicon.ico'
'my-react-app/build/index.html' -> 'website/index.html'
'my-react-app/build/manifest.json' -> 'website/manifest.json'
'my-react-app/build/precache-manifest.16ab4e73d949e217b5bb10bd22a96200.js' -> 'website/precache-manifest.16ab4e73d949e217b5bb10bd22a96200.js'
'my-react-app/build/service-worker.js' -> 'website/service-worker.js'
'my-react-app/build/static' -> 'website/static'
'my-react-app/build/static/media' -> 'website/static/media'
'my-react-app/build/static/media/logo.5d5d9eef.svg' -> 'website/static/media/logo.5d5d9eef.svg'
'my-react-app/build/static/css' -> 'website/static/css'
'my-react-app/build/static/css/main.8cc6a47e.chunk.css' -> 'website/static/css/main.8cc6a47e.chunk.css'
'my-react-app/build/static/css/main.8cc6a47e.chunk.css.map' -> 'website/static/css/main.8cc6a47e.chunk.css

In [65]:
aws s3 rm --recursive s3://mjbright-static-site/

delete: s3://mjbright-static-site/manifest.json
delete: s3://mjbright-static-site/index.html
delete: s3://mjbright-static-site/favicon.ico


In [66]:
aws s3 ls s3://mjbright-static-site/

In [67]:
aws s3 sync website/ s3://mjbright-static-site/

upload: website/asset-manifest.json to s3://mjbright-static-site/asset-manifest.json   
upload: website/precache-manifest.16ab4e73d949e217b5bb10bd22a96200.js to s3://mjbright-static-site/precache-manifest.16ab4e73d949e217b5bb10bd22a96200.js
upload: website/favicon.ico to s3://mjbright-static-site/favicon.ico                 
upload: website/static/css/main.8cc6a47e.chunk.css.map to s3://mjbright-static-site/static/css/main.8cc6a47e.chunk.css.map
upload: website/static/media/logo.5d5d9eef.svg to s3://mjbright-static-site/static/media/logo.5d5d9eef.svg
upload: website/static/js/main.a7b1c78b.chunk.js.map to s3://mjbright-static-site/static/js/main.a7b1c78b.chunk.js.map
upload: website/index.html to s3://mjbright-static-site/index.html 
upload: website/service-worker.js to s3://mjbright-static-site/service-worker.js
upload: website/static/js/runtime~main.229c360f.js.map to s3://mjbright-static-site/static/js/runtime~main.229c360f.js.map
upload: website/static/css/main.8cc6a47e.chunk.css t

We can now test our new website: using wget, curl or http(ie) or using a browser at http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html


In [68]:
http http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 2062
Content-Type: text/html
Date: Fri, 25 Jan 2019 22:15:15 GMT
ETag: "7919b1a9089a669e063517d00d7ada31"
Last-Modified: Fri, 25 Jan 2019 22:15:11 GMT
Server: AmazonS3
x-amz-id-2: 5EIMuAzMksTQxTlEfSa5l6oiCb3YxMHZzEAtZtLAnl1osQt6/G4oQmIaDgpep/4Q65udFtk2P+0=
x-amz-request-id: 9483B6104E576BD8

<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="shortcut icon" href="/favicon.ico"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"/><meta name="theme-color" content="#000000"/><link rel="manifest" href="/manifest.json"/><title>React App</title><link href="/static/css/main.8cc6a47e.chunk.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(l){

# Further Work

## S3 Static Site Hosting

An article describing the use of S3 for static website hosting including use of https, DNS routing
https://medium.freecodecamp.org/how-to-host-a-static-website-with-s3-cloudfront-and-route53-7cbb11d4aeea


## Static Site Generators

Try other Static Site generators such as Hugo, Gatsby or Jekyll.

Information about such generators is available here: https://www.staticgen.com/



You can find more details about S3 website hosting here: https://docs.aws.amazon.com/AmazonS3/latest/dev/WebsiteHosting.html


# Cleanup

Note that we can use the ```aws s3 rm``` command to remove files from the bucket and ```aws s3 rb``` command to remove a bucket.

In [ ]:
aws s3 rm s3://mjbright-static-site --recursive
aws s3 rb s3://mjbright-static-site

It's also possible to remoce the bucket directly using the ```--force``` option:
    ```aws s3 rb --force s3://mjbright-static-site```